# Appendix B: Problem 9

Given information - Components names, Molecular Weights & Molar Fractions

In [2]:
# Setup problem constraints
R = 10.73146 # Universal gas constant, psia-ft3/R-lbm mol
pr = 5200 #psia
tr = 250 #deg F
psp1 = 950 #psia
tsp1 = 160 #deg F
qg_prim = 4265 #mscf/d
sg_prim = 0.70
qo_tank = 370 #stb/d
api_tank = 45

In [26]:
# Solution

# First step - Estimating additional gas in seperator oil and its specific gravity
A1 =((psp1/18.2+1.4)*10**(0.0125*api_tank-0.00091*tsp1))**1.205 # Eq 3.61b
A2 = 0.25+0.02*api_tank                                         # Note the worked example equation is incorrect, it should be 0.02*api_tank (per the original equation in Chapter 3)
A3 = -3.57e-6*api_tank
Rsplus = A1*A2/(1-A1*A3)
sg_tank = A2 + A3*Rsplus
R1 = qg_prim*1e3/qo_tank

# Then total gas properties
Rp = R1 + Rsplus   # scf/stb
rp = 1e6/Rp        # stb/mmscf
gamma_g = ((sg_prim*R1)+(sg_tank*Rsplus))/(R1+Rsplus) # total gas sg
print('Total gas sg:',gamma_g )

Total gas sg: 0.7133829933038295


In [29]:
# Use Cragoe correlation to estimate stock tank oil MW
Mo = 6084/(api_tank-5.9)

# and full wellstream gas properties
gamma_o = 141.5/(131.5+api_tank)
gamma_fws = (gamma_g+4580*rp*1e-6*gamma_o)/(1+133000*rp*1e-6*(gamma_o/Mo)) # Eq 3.55
print('Stock tank oil MW:',Mo)
print('Stock tank oil sg:', gamma_o)
print('Full wellstream sg:', gamma_fws)

Stock tank oil MW: 155.6010230179028
Stock tank oil sg: 0.8016997167138811
Full wellstream sg: 0.9654026539884318


In [34]:
# Sutton pseudocritical correlations
Tpc = 169.2+349.5*gamma_fws-74.0*gamma_fws**2
Ppc = 756.8-131.0*gamma_fws-3.6*gamma_fws**2

Tpr = (tr+460)/Tpc
Ppr = pr/Ppc

In [37]:
# Copying Hall-Yarborough Z factor solution from Problem 6
from scipy.optimize import fsolve
import math
def fy(y, *args):
    alpha, Pr, t = args
    return -alpha * Pr + (y + y ** 2 + y ** 3 - y ** 4) / (1 - y) ** 3 - (14.76 * t - 9.76 * t ** 2 + 4.58 * t ** 3) * y ** 2 + (90.7 * t - 242.2 * t ** 2 + 42.4 * t ** 3) * y ** (2.18 + 2.82 * t)
    
def Zfac(Tr, Pr):
    t = 1 / Tr
    alpha = 0.06125 * t * math.exp(-1.2 * (1 - t) ** 2)
    args = (alpha, Pr, t)
    y = 0.001
    y = fsolve(fy, y, args=args)
    return alpha * Pr / y

In [51]:
Z = Zfac(Tpr, Ppr)[0]
Cog = 133000*(gamma_o/Mo) # scf/stb
# **Note, in following evaluation, I think that the example has incorrectly used seperator temperature instead of reservoir temperature
Bgd = (14.7/520)*Z*(tr+460)/pr*(1+Cog/Rp) #Dry gas FVF = res gas vol / surface gas volume 

#Initial daily voidage
delta_Vg = qo_tank * R1 * Bgd
print('Initial daily voidage:', round(delta_Vg,1), 'cuft =', round(delta_Vg/5.614583,1), 'bbls')

Initial daily voidage: 17823.2 cuft = 3174.4 bbls
